### Data
Please see jupyter notebook tilted "NHL_attendance_scrape" to see the code for scraping the attedance of every regular season game from the 2018-19 season from Hockey Reference. I chose to analysis the 2018-19 season becuase that was the last full NHL season unaffected by Covid restrictions.

x data points = Home Game

y data points = Attendance

Source: https://www.hockey-reference.com/leagues/NHL_2019_games.html

*Note: A model for the Tampa Bay Lightning, Boston Bruins, Edmonton Oilers, and Winnipeg Jets will not generated from this dataset because according to Hockey Reference, they sold out every single home game each. Congrats to them, but there is no model for a vertical line*


### Structure:
1) Model and Error Propagation Function: 
- The first six functions define the linear, power and parabolic models and their error propagation formulas. These error propagation formula are used to generate the confidnce bands (CB) of the fit

2) Plot type functions:
- *Curve_Fit*: Use scipy to determine the best fit regression model base on given data. Plot the x and y data point with regression line and display the best fit coefficients. Call according error propagation function to plot CB. Plot the prediction band (using MSE). Calculate and display regression evaluation metrics eg, SSE, RMSE, R^2 (linear only) 
- *Resolution_plot*: Calculate the width of the CB at every point along the x axis to determine the prediction resolution at each point. I expect most of these plots to be empty becuase to calculate the width of the CB you should be able to draw a horizontal line intersecting both the upper and lower CB. However, there is so much variance in these regression fits, I doubt this will happen.
- *Residual_Distribution_Plot*: Plot the residuals between the acutal attendance and the predicted attendnace vs the predicted attendance with the centre horizontal line represnted the regression line. There should be a normal distribution of resuduals surrounding the regression line. This distribution is indicated by a vertical histogram that will be sitting just right to the residual plot
- *Russian_alphabet_plot*: AKA Kolmorgov-Smirnov goodness-of-fit test. The orange line is the Culmative Distribution Funcion (CDF) of a normal distribution and blue line is the empherical CDF of the selected regression line. A significance test can be performed on the separation between the normal and emphericla CDF. Where is the p-value is less than 0.05, we can condclude that the regression fit is not good. In practice this test has a low pass criteria and should be modified to be more strict.

3) Create master figure:
- Use the library gridspec to order all the plots onto on page per dataset.
- Order of the plot on one is page is as follows:
    - Top Left: Curve Fit Plot
    - Top Right: Resolution Ploy
    - Bottom Left: Residual Distribution Plot
    - Bottom Middle: Histogram of Residual Ditsrubution 
    - Bottom Right; CDF plot and Kolmorgov-Smirnov Test

4) Call All Functions
- Import NHL_attendance data
- Create excel to store all performance evaluation metrics
- Loop through each team to create a subdataset to run through model generation and plotting
- Save all figures to pdf (Ecpect 87 different pdfs. 29 teams x 3 model types)

Overall, some sections of the code may seem inefficent. This is becuase these script was originally designed to hdanle datasets with much more parameters and user input selection

**Scroll to the bottom to see data visulaization output and final conclusions**

In [160]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Model and Error Propagation Functions
- power_model()
- power_err_prop()
- linear_model()
- linear_err_prop()
- parabolic_model()
- parabolic_err_prop()

In [282]:
import numpy as np
from matplotlib import pyplot as plt
from uncertainties import ufloat
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from scipy.stats import kstest
import scipy.stats as stats
import random
import math
from scipy.optimize import least_squares
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib as mpl
from matplotlib import gridspec

In [283]:
def power_model(x, alpha = 1, beta = 1, gamma = 0):
    '''
    Purpose: 
        Map an argument, x, to a response, y, with a power law relationship given by:   f(x) = y = coef*x^power + inercept
        
    Input:
        x         (float - array) : The argument for the power law function
        coef      (float)         : the coefficient in the power law function
        power     (float)         : the exponent of the power law function
        intercept (float)         : the intercept of the power law function
    Output:
        y         (float)         : The response of the argument, x, under the power law function/mapping
    '''
    
    y = alpha*(x**beta) + gamma
    return y


In [284]:
def power_err_prop(x, alpha, beta, gamma, cov):
    '''
    Purpose:
        To compute the error propagation of a power-law curve. The terms use the typical error propagation formula, where each term is a product of partial derivatives
        multiplied by the corresponding covariance term.
        
        The power-law function takes the form:   y = alpha*x^beta + gamma
        
    Input:
        x     (array/float)    : argument to the power law function
        alpha (float)          : alpha parameter (or its estimate) of the power-law function above
        beta  (float)          : beta parameter (or its estimate) of the power-law function above
        gamma (float)          : gamma parameter (or its estimate) of the power-law function above
        cov   (2D array/float) : the covariance matrix of the three input parameters alpha, beta, gamma
        
    Output:
        var_prop (array/float) : propagated error evaluated at the input argument
    '''
    
    # Diagonal terms of the error prop:
    
    term_1 = np.power(x, 2*beta)*cov[0][0]
    term_2 = np.power(alpha*np.log(x), 2.)*np.power(x, 2.*beta)*cov[1][1]                  
    term_3 = cov[2][2]
    
    # Mixed terms of the error prop:
    
    term_4 = 2*alpha*np.power(x,2.*beta)*np.log(x)*cov[0][1]
    term_5 = 2*np.power(x, beta)*cov[0][2]
    term_6 = 2*alpha*np.power(x, 3.*beta)*np.log(x)*cov[1][2] 
    
    var_prop = term_1 + term_2 + term_3 + term_4 + term_5 + term_6
    
    return var_prop

In [285]:
def linear_model(x, alpha, beta):
    '''
    Purpose: 
        Map an argument, x, to a response, y, with a power law relationship given by:   f(x) = y = alpha*x + beta
        
    Input:
        x         (float - array) : The argument for the power law function
        alpha     (float)         : the slope in the linear model
        beta      (float)         : the intercept in the linear model
       
    Output:
        y         (float)         : The response of the argument, x, under the linear model
    '''
    
    y = alpha*x + beta
    return y

In [286]:
def linear_err_prop(x, alpha, beta, cov):
    
    '''
    Purpose:
        To compute the error propagation of a linear curve. The terms use the typical error propagation formula, where each term is a product of partial derivatives
        multiplied by the corresponding covariance term.
        
        The linear function takes the form:   y = alpha*x + beta
        
    Input:
        x         (float - array) : The argument for the power law function
        alpha     (float)         : the slope in the linear model
        beta      (float)         : the intercept in the linear model
        cov   (2D array/float) : the covariance matrix of the three input parameters alpha, beta, gamma
        
    Output:
        var_prop (array/float) : propagated error evaluated at the input argument
    '''
    
    
    import numpy as np
    
    # Diagonal terms of the error prop:
    term_1 = np.power(x, 2)*cov[0][0]
    term_2 = cov[1][1]
    
    # Mixed terms of the error prop:
    term_3 = 2*x*cov[0][1]
    
    var_prop = term_1 + term_2 + term_3 
    
    return var_prop

In [287]:
def parabolic_model(x, alpha, beta, gamma):
    '''
    Function : parabola
    
    Purpose  : Apply a parabolic function to the input argument, x, where parabola(x) = alpha*x^2 + beta*x + gamma
    
    Input: 
        x (float/array) : an array of floats to be used as an argument for the parabolic function 
        alpha (float)      : the coefficient for the second order term of the parabola 
        beat (float)      : the coefficient for the first order (linear) term of the parabola
        gamma (float)      : the zeroth order (constant) term of the parabola
        
    Output:
        y (float/array) : The mapping of the argument, x, under the parabolic function
    '''
    
    t2 = alpha*(x**2.)
    t1 = beta*x
    
    y = gamma + t1 + t2


    return y

In [288]:
def parabola_error_prop(x, alpha,beta,gamma, cov):
    '''
    Purpose:
        To compute the error propagation of a parabola whose coefficients have been estimated with some uncertainty. The terms use the typical error propagation formula, where each term is a product of partial derivatives
        multiplied by the corresponding covariance term.
        
        The parabola function takes the form:   y = alpha*x^2 + beta*x + gamma
        
    Input:
        x   (array/float)    : argument to the power law function
        gamma  (float)          : zeroth order coeffcient (or its estimate) of the parabolic function above
        beta  (float)          : first order coefficient (or its estimate) of the parabolic function above
        alpha  (float)          : second order coefficient (or its estimate) of the parabolic function above
        cov (2D array/float) : the covariance matrix of the three input parameters p0, p1, p2
        
    Output:
        varprop (array/float) : propagated error evaluated at the input argument
    '''
    import numpy as np    
        
    # diagonal terms:
    t1 = np.power(x, 4.)*cov[0][0]
    t2 = np.power(x, 2.)*cov[1][1]
    t3 = cov[2][2]
    
    # mixing terms:
    t4 = 2*np.power(x, 3.)*cov[0][1]
    t5 = 2*np.power(x, 2.)*cov[0][2]
    t6 = 2*x*cov[1][2]
    
    # Combine for total variance:
    varprop = t1 + t2 + t3 + t4 + t5 + t6
    
    return varprop

## Plot types Functions
- curvefit_plot
- resolution_plot
- residual_distribution_plot
- russian_alphabet_plot

In [289]:
def curvefit_plot(x_data, y_data, model,error_prop_model, model_choice, ax):
#Best fit estimates and plot the curve with CB and PB
    random.seed(12345)
    
    #Perform curve fit 
    bestest, covar = curve_fit(model, x_data, y_data)
    sigma_est  = np.sqrt(np.diagonal(covar))
 
    #Calculate residuals and stats
    d_f = len(bestest)
    
    
    predictions = model(x_data, *bestest)
    res = y_data - predictions
    sse = np.sum(res**2) #this is SSE 
    mse = sse/(len(y_data)-d_f)
    rmse = math.sqrt(mse)
    
    ss_tot = np.sum((y_data-np.mean(y_data))**2)
    r_squared = 1 - (sse / ss_tot)

    n_ypoints = len(y_data)

    predband_error = mse
    

    #Plotting
    ####################################################################
    #fig, ax = plt.subplots(figsize = (16.18, 10))
    
    band_x = np.linspace(min(x_data), max(x_data), 300)  
    
    predictions_plot = model(band_x, *bestest)
    
    # plot the data points:
    ax.scatter(x_data, y_data, facecolor = 'silver',edgecolor = 'k', s = 10, alpha = 1)
    
    # plot the model:
    ax.plot(band_x, predictions_plot, 'black') # regression line
    
    #Plot the CB 
    bound_upper = predictions_plot + scipy.stats.t.ppf(0.975, n_ypoints-d_f)*np.sqrt(error_prop_model(band_x, *bestest, covar)) # upper edge of confidence band
    bound_lower = predictions_plot - scipy.stats.t.ppf(0.975, n_ypoints-d_f)*np.sqrt(error_prop_model(band_x, *bestest, covar)) # lower edge of confidence band
    bound_lower = np.array(bound_lower, dtype=float)
    bound_upper = np.array(bound_upper, dtype=float)
    
    ax.plot(band_x, bound_upper, linewidth = 2, alpha = 0.4, color = 'cyan')
    ax.plot(band_x, bound_lower, linewidth = 2, alpha = 0.4, color = 'cyan')
    ax.fill_between(band_x, bound_lower, bound_upper, color = 'cyan', alpha = 0.1, label = 'Confidence Band') 

    # Add prediction band edges:
    pred_bound_upper = predictions_plot + scipy.stats.t.ppf(0.975, n_ypoints-d_f)*np.sqrt(error_prop_model(band_x, *bestest, covar) + predband_error) # upper edge of confidence band
    pred_bound_lower = predictions_plot - scipy.stats.t.ppf(0.975, n_ypoints-d_f)*np.sqrt(error_prop_model(band_x, *bestest, covar) + predband_error)
    ax.plot(band_x,pred_bound_upper, color = 'green', linestyle = '--', linewidth = 1, label = 'Prediction Band')
    ax.plot(band_x,pred_bound_lower, color = 'green', linestyle = '--', linewidth = 1)

    # Axes labels and title:
    ax.set_ylabel('Attendance', fontsize=16, rotation = 'vertical')
    #ax.yaxis.set_label_coords(-0.07,0.5)
    ax.set_xlabel('Home Game', fontsize = 16)
    ax.set_title('Curve Fit', fontsize = 20)
    

    # Adjust the frame of the plot:
    ax.spines['left'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['right'].set_color('black')
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(alpha = 0.2)
    
    
    # place a text box in upper left in axes coords:
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    
    sigma_est  = np.sqrt(np.diagonal(covar))
    list_of_coeff =['alpha','beta','gamma']
    
    coeff_output = ''
    for coeff in range(len(bestest)):
        coeff_estimate = '{} = {}\n'.format(list_of_coeff[coeff],ufloat(bestest[coeff], sigma_est[coeff]) )
        coeff_output = coeff_output + coeff_estimate
        
    ax.text(0.05, 0.95, 'Best Fit Coefficients\n--------------------------------\n'+coeff_output, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    

    if model_choice == '1': #If linear model, use r^2 value   
        sse_r2_text = np.round(r_squared,4)
    else: #If Parabolic or power use sse value
        sse_r2_text = np.round(sse,4)
        
    ax.text(0.05, 0.7, 'RMSE: {} \nSSE or R^2: {} \nn = {}'.format(np.round(rmse,4) ,  sse_r2_text, n_ypoints), transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
    
    # annnnnd the legend:
    ax.legend(loc = 'lower right', fontsize = 16)

    
    return bestest, predband_error, covar, res, predictions, sse_r2_text, rmse, coeff_output
    

In [290]:
def resolution_plot (model, err_prop_model, bestest, covar, predband_error, y_data,x_data, ax):
#Calculate the relative resolution and plot    
    n_ypoints = len(y_data)
    d_f = len(bestest)
    
    current_range = np.linspace(min(y_data)*0.9, max(y_data)*1.1, 1000)
    from scipy.optimize import fsolve
    band_conc = np.linspace(-10, 10, 10000)
    spreads = []
    estimates = []
    
    
    initial_guess = (x_data.min() + x_data.max())/2
    
    
    for current in current_range:
        def best_model_upper(x):
            return model(x, *bestest) + scipy.stats.t.ppf(0.975, n_ypoints-d_f)*np.sqrt(err_prop_model(x, *bestest, covar) + predband_error) - current 

        def best_model_lower(x):
            return model(x, *bestest) - scipy.stats.t.ppf(0.975, n_ypoints-d_f)*np.sqrt(err_prop_model(x, *bestest, covar) + predband_error) - current 

        def best_model(x):
            return model(x, *bestest) - current 

        
        
        root_low = least_squares(best_model_upper, bounds = (x_data.min(), x_data.max()), x0 = initial_guess)
        root_low_x = root_low.x
        root_low_cost = root_low.cost
        # the second argument is just an initial guess that is about midway through our range and is also                                                                                         
        # recognizable as the default result due to its obvious pattern.
        root_upp = least_squares(best_model_lower, bounds = (x_data.min(), x_data.max()), x0 = initial_guess)
        root_upp_x = root_upp.x
        root_upp_cost = root_upp.cost
        
        cost_limit = 10**(-10)
        
        estimate = least_squares(best_model, bounds = (x_data.min(), x_data.max()), x0 = initial_guess).x    
        
        #Check any case the cost of least squares is gretaer then limit. This means there is likely no proper real solution 
        if ((root_upp_cost > cost_limit) | (root_low_cost > cost_limit)):
            continue
            
        else:      
            estimates.append(estimate)
            spread = root_upp_x - root_low_x
            spreads.append(spread/estimate*100)


    #plot the labels, target resolution and spines of the resolution plot
    
    target_resolution = 10
    

    ax.set_ylabel('Resolution(%)', fontsize=16, rotation = 'vertical')

    ax.set_xlabel('Home Game', fontsize = 16)
    
    ax.set_title('Relative Resolution (Width of CI / Estimate)', fontsize = 20)
    
    ax.spines['left'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['right'].set_color('black')
    ax.tick_params(axis='both', labelsize=14)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    
    
    
    #Create a case where the estimates is empty 
    
    if not estimates:
        final_text = 'No intersection between the upper and lower prediction bands'
        ax.text(0.05, 0.95, final_text, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

    else: 
        ax.scatter(estimates, spreads, s = 3)
        res_goal = list(filter(lambda x : x[0]<10 ,spreads))
        
        max_y = max(spreads)*1.05
        max_x = max(estimates)*1.05
        min_x = min(estimates)*0.95
        
        ax.axvline(x=estimates[np.argmin(spreads)], ymin=0, ymax = min(spreads)/max_y, linewidth = 3, c = 'pink')

   
    
        ax.set_xlim(min_x, max_x)
    
       
        final_text = 'Min Res (%)'.format(np.round(min(spreads)[0], 2), np.round(estimates[np.argmin(spreads)][0],2))
    

        
        ax.text(0.05, 0.95, final_text, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)

        
        
    ax.grid(alpha = 0.2)
    
    
    
    return final_text

In [291]:
def residual_distribution_plot(res, predictions, ax1, ax2):
#Create the residual plot and residual histogram

    num_bins = 15

    x_bins = (predictions.max()-predictions.min())/num_bins
    
    start = predictions.min()
    full_bins=[]
    center_list=[]
    
    #This for loop will calculate the sum or residuals vs. Concentration line trend
    #Maybe build in the user option to turn this line on or off. Dont know how useful it can be
    for b in range(num_bins+1):
        
        end = start+x_bins
        center = start+x_bins/2
        bin_i = []
        
        for i in predictions.index:
            
            if start <= predictions[i] <= end:
                
                bin_i.append(res[i])
        
        full_bins.append(np.sum(bin_i))  
        center_list.append(center)
        start = end
    
    #Plot the residual plot on ax1 
    ax1.scatter(predictions, res)
    ax1.axhline(0, c = 'black', linewidth = 3, label = 'Regression Line')
    #ax1.plot(center_list, full_bins, color = 'green', lw=2,alpha = 0.6, label ='Sum of Residuals Plot')
    ax1.set_xlabel('Attendance Estimates',fontsize=16)
    ax1.set_ylabel('Residuals', fontsize=16)
    ax1.legend(loc = 'upper right', fontsize = 16)
    ax1.set_title('Residual Plot', fontsize = 20)
    ax1.spines['left'].set_color('black')
    ax1.spines['top'].set_color('black')
    ax1.spines['bottom'].set_color('black')
    ax1.spines['right'].set_color('black')
    ax1.tick_params(axis='both', labelsize=14)
    ax1.grid(visible=True, alpha = 0.2)
    
    #Plot the residual histogram on ax2
    ax2.hist(res, orientation = 'horizontal', edgecolor = 'black', lw=1)
    ax2.spines['left'].set_color('black')
    ax2.spines['top'].set_color('black')
    ax2.spines['bottom'].set_color('black')
    ax2.spines['right'].set_color('black')
    ax2.tick_params(axis='both', labelsize=14)
    
    
    return 

In [292]:
def russian_alphabet_plot(res,ax):
#Perform K-s test and graoh residuals cdf vs a normal cdf

    #Standardize the residuals
    mean = np.mean(res)
    std = np.std(res)
    
    stand_res =[]
    for x in res:
        z = (x - mean)/std
        stand_res.append(z)
    
    #Calculate the residuals cdf
    x_res = np.sort(stand_res)
    res_cdf = np.arange(len(stand_res))/len(stand_res)
    
    #Calcuate the normal cdf
    x_range = np.linspace(-3,3,1000)
    normal_cdf = stats.norm.cdf(x_range, 0, 1)
    
    ax.plot(x_res, res_cdf,marker='o', label = 'Residuals CDF')
    ax.plot(x_range ,normal_cdf, label = 'Normal CDF')
    ax.set_xlim(-3,3)
    ax.set_title('CDF Plot', fontsize =20)
    ax.set_ylabel('Probability', rotation ='vertical', fontsize = 16)
    ax.set_xlabel('Residuals', fontsize =16)
    ax.tick_params(axis='both', labelsize=14)
    
    ax.legend(loc = 'lower right', fontsize = 16)
    
    #Perfrom K-S test
    result = kstest(stand_res,'norm')
    
    p_value = 0.05
    
    if result[1] > p_value:
        ks_text = 'P-Value: {} \nK-S Test: Pass'.format(np.round(result[1],3))
    else:
        ks_text = 'P-Value: {} \nK-S Test: Fail'.format(np.round(result[1],3))
        
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.05, 0.95, ks_text, transform=ax.transAxes, fontsize=18,verticalalignment='top', bbox=props)
    ax.grid(visible=True, alpha =0.2)
    
    return ks_text

## Create Master Figure using GridSpec

In [293]:
def master_figure(x_data,y_data, model, title):
#Define the dictionaries to call each model and create the master figure using gridspec

    model_dict = {'1' : linear_model, '2': power_model, '3':parabolic_model} 
    error_prop_dict = {'1' : linear_err_prop, '2': power_err_prop, '3':parabola_error_prop}
    
    reg_model = model_dict[model]
    error_prop_model = error_prop_dict[model]
    
    master_fig = plt.figure(figsize=(30,20))
    spec = mpl.gridspec.GridSpec(ncols=6, nrows=2,wspace=0.25,hspace=0.2) # 6 columns evenly divides both 2 & 3

    ax1 = master_fig.add_subplot(spec[0,0:3]) # row 0 with axes spanning 2 cols on evens
    ax2 = master_fig.add_subplot(spec[0,3:6])
    ax3 = master_fig.add_subplot(spec[1,0:2])
    ax4 = master_fig.add_subplot(spec[1,2:3], sharey = ax3)# row 0 with axes spanning 2 cols on odds
    ax5 = master_fig.add_subplot(spec[1,3:6])

    #Executing all graphing functions
    bestest , predband_error , covar, res, predictions, sse_r2_text,rmse, coeff_output  = curvefit_plot(x_data, y_data, reg_model, error_prop_model, model, ax1)
    resolution = resolution_plot(reg_model,error_prop_model, bestest, covar, predband_error, y_data, x_data, ax2)
    residual_distribution_plot(res, predictions,ax3, ax4)
    ks_text = russian_alphabet_plot(res,ax5)
    
    master_fig.suptitle( title, fontsize = 26, y=0.92)
    
    plt.show(master_fig)
    
    return master_fig, ks_text, coeff_output, sse_r2_text, rmse , resolution


## Call All Functions
- import NHL_attendance data
- create excel to store all performance evaluation metrics
- loop through each team to create a subdataset to run through models generation
- Save all figures to pdf (Expect 90 different pdfs. 30 teams x 3 model types)

In [294]:
def curve_evaluation_main():
    
    model_names = {'1' : 'Linear Model', '2': 'Power Model', '3':'Parabolic Model'}
    model_table = ['Linear Model', 'Power Model', 'Parabolic Model']
    model_selection = ['1', '2', '3']
    
    evaluation = ['K-S Test' ,'Coeificents','SSE/R^2','RMSE','Resolution' ]
    

    
    attendance_data = pd.read_csv('../NHL attendance/NHL_attendance_df.csv')
    attendance_data['Att.'] = attendance_data['Att.'].apply(lambda x: float(x.replace(',','')))
    attendance_data['Home Game'] = attendance_data['Home Game'].apply(lambda x: float(x))
    #Now setup the output table                               
    ##############################################################################    
    
    attributes = list(attendance_data['Home'].unique())
    
    #Create multi-index lables for output tables
    column_labels = pd.MultiIndex.from_product([model_table, evaluation])
    index_labels = pd.MultiIndex.from_product([attributes])

    curve_metrics = pd.DataFrame(columns = column_labels, index = index_labels)
                            
    #Perform curve fitting
    ######################################################
    fig_list =[]

    for team in attributes:
        
        team_data = attendance_data[attendance_data['Home'] == team]     
        if len(team_data['Att.'].unique()) != 1:
            for model in model_selection:
                
                                 
                title = 'Dataset: {} - {}'.format(team,model_names[model])
                
                y_data = team_data['Att.']
                x_data = team_data['Home Game']            

            
                try:
                    master_fig, ks_test, coeff_output, sse_r2_text, rmse , resolution =  master_figure(x_data, y_data, model, title)
            
                    curve_metrics.loc[team,(model_names[model],'K-S Test')]= ks_test             
                    curve_metrics.loc[team,(model_names[model],'Resolution')]= resolution
                    curve_metrics.loc[team,(model_names[model],'SSE/R^2')]= sse_r2_text
                    curve_metrics.loc[team,(model_names[model],'RMSE')]= rmse
                    curve_metrics.loc[team,(model_names[model],'Coeificents')]= coeff_output
                
                    fig_list.append(master_fig)
                    
                
                except RuntimeError:  
                    print ('No solution {}, - {}'.format(model_names[model],team))  
        else:
            print('Team {} has sold out every single game. There is resulting model'.format(team))
    
    #Save graph plus output table
    #This probably should be its own function
    ####################################################
    
    
    title_main = 'NHL_attendnace_Curve_Fit'
        
    pp = PdfPages('../NHL attendance' + '/' + title_main + '.pdf')
    for i in fig_list:
        pp.savefig(i)
    pp.close()
    
    curve_metrics.to_excel('../NHL attendance' + '/Curve_Metrics.xlsx' )
    
    
    
    return  


## Run the Code

### Please view visualization output in 'NHL_attendance_vis' PDF.  

In [1]:
#curve_evaluation_main()

## Conclusions

As expected none of these regression model fit the attendance data well. 

It can be seen that the confidence bands flair our dramatically. This is becuase the derivateion of the error propagation forumla for power model has large exponetial term that expand quickly when fitting data that does follow a power law. In some cases the power model didn't even converge to a solution

In any case, I will demostrate how you could interpret these plots and metrics to efficiently evlauate and compare the models.

**Example - St.Louis Blues Parabolic Model**
1) Check if a parabolic model accurate describes the realationship in this dataset. Tools to use: Curve Plot, residuals plot and CDF plot
- First we examine the Curve fit of the linear model to qualitativley see if the regression line is passing through the centre of the distribution of points, whcih appears to be in this case. We can also qualitatively examine the behavoir of the Confidence and Prediction Bands
- Next look at the residual plot to see if the distribution of residuals around the regression line is normal. This affirms the regression model is accurately pasisng through the middle of the distribution of points. Thus we can conclude a parabolic model is appropraite for this datatset. 
- The CDF plot demonstarte that the CDF of the model accurate follows the CDF of a normal distribution, thus passing the Kolmorgov-Smirnov test. A final check to affirm a paraboloc fit is correct for this data

2) Compare the quality of this fit to other models. Tools to use: Resolution Plot, Curve Metrics RMSE, SEE and R^2, Uncertancy of Best Fit Coefficients. These values are also stored and eaily comparable in the excel file 'Cuve_Metrics' 
- Resolution plot can be used to compare the width of CB between differnt models correlating to the uncertancy of the regression fit. However, this tool cannot be used in this case becuase no model has stable enough CB wehere a horizontal line crosses both the upper and lower CB
- The uncertanicy (+/-) On the best coeifficent can also be used to evlaute the uncertancy of the regression fit.
- Metrics such as RMSE and SSE explain the error between the actual attendnace values and the regression line. The smaller the emtric indicates a better fit. RMSE can be used to compare between Linear, Parabolic and Power model, while SSE can only be compared between Parabolic and Power model. 
- R^2 is a more robost goodness-of-fit test for linear model. Where the closer r^2 is to 1.0 indicated a linear fit that describes all the varinace in the dataset, this value can nly be used to compare R^2 values. 
- However, these mtrics could be interpreted in different wasy based on the domain and application of the data
- In this case, the St.Louis Blues Parabolic Model has middle of the pack RMSE and SSE valus. So maybe this fit is lower quality then originally thought

3) Back to Reality

Why would the St.Louis Blues 2018-19 season regular season home attendance follow an upwards parabolic model?

This was the year the Bluse went from last place in the NHL on January 1st, to winning the Stanely Cup on the back of breakout goaltender Jordan Binnington
Pretty cool we can see this in the data!

_________________

I hope this demonstrated how useful this data visulization tool *can* be when generating and comparing regression fits for data that actually closly reflect a linear, parabolic or power model. This script provides the tools to select a superior model from fits that seemingly have very little difference. 

Thanks for visiting my project!!